In [1]:
import cma_gui as cma
import pandas as pd

# Run All Calcs

In [2]:
# Check if new data needs to be pulled
date_check = pd.read_excel('P:\\Advisory\\Research\\Automation\\CMAs\\Data\\bloomberg_data_usd.xlsx', index_col=0)

if (cma.val_dict['as_of_date'] in date_check.index):
    import backfill_calc
    import std_dev
#    import fixed_income_calcs
    import equity_calcs
    import alts
    print('values were same')
else:
    import data_pull
    import backfill_calc
    import std_dev
#    import fixed_income_calcs
    import equity_calcs
    import alts
    print('values were different')

values were same


# Final Data

## US

In [3]:
df_final_us = pd.concat([equity_calcs.equity_returns_us, alts.alts_returns_us])
df_final_us = pd.DataFrame(df_final_us)

# Combine Expected Returns with Standard Deviation
df_final_us = df_final_us.merge(std_dev.annual_adj_std_dev_us*100, left_index=True, right_index=True)
df_final_us = df_final_us.rename(columns={0: "Standard Deviation"})

# Convert to ints and round
df_final_us['Expected Return'] = pd.to_numeric(df_final_us['Expected Return']).round(1)
df_final_us['Standard Deviation'] = pd.to_numeric(df_final_us['Standard Deviation']).round(1)

# Reorder
equity_us_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'equity_us_name' in k}.values())))
#fixed_us_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'fixed_us_name' in k}.values())))
alts_us_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'alts_us_name' in k}.values())))

order_name = equity_us_name + alts_us_name

df_final_us = df_final_us.reindex(index=order_name)
df_final_us

,Expected Return,Standard Deviation
U.S. Equity,6.3,14.9
U.S. Large Cap Equity,6.2,14.7
U.S. Mid Cap Equity,6.6,16.5
U.S. Small Cap Equity,7.0,19.6
U.S. Micro Cap Equity,6.9,21.0
Global Equity,6.5,15.1
International Developed Equity,6.2,16.5
International Small Cap Equity,6.2,17.6
Global Emerging Markets Equity,8.5,21.2
Absolute Return,4.6,5.2


## NonUS

In [5]:
df_final_nonus = pd.concat([equity_calcs.equity_returns_nonus, alts.alts_returns_nonus])
df_final_nonus = pd.DataFrame(df_final_nonus)

# Combine Expected Returns with Standard Deviation
df_final_nonus = df_final_nonus.merge(std_dev.annual_adj_std_dev_nonus*100, left_index=True, right_index=True)
df_final_nonus = df_final_nonus.rename(columns={0: "Standard Deviation"})

# Convert to ints and round
df_final_nonus['Expected Return'] = pd.to_numeric(df_final_nonus['Expected Return']).round(1)
df_final_nonus['Standard Deviation'] = pd.to_numeric(df_final_nonus['Standard Deviation']).round(1)

# Reorder
equity_nonus_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'equity_nonus_name' in k}.values())))
#fixed_us_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'fixed_us_name' in k}.values())))
alts_nonus_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if 'alts_nonus_name' in k}.values())))

order_name_nonus = equity_nonus_name + alts_nonus_name
df_final_nonus = df_final_nonus.reindex(index=order_name_nonus)

df_final_nonus

,Expected Return,Standard Deviation
U.S. Equity,6.2,14.6
U.S. Small Cap Equity,6.7,19.2
Europe Ex-UK Equity,6.5,16.8
Europe Small Cap Equity,6.3,17.9
UK Equity,7.0,13.1
Japan Equity,4.4,15.0
Japan Small Cap Equity,4.3,16.8
Developed Market Pacific Ex-Japan Equity,8.2,18.3
Global Emerging Markets Equity,8.4,19.7
Long-Short Equity,5.7,9.3
